In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        break

/kaggle/input/intel-image-classification/seg_train/seg_train/mountain/14986.jpg
/kaggle/input/intel-image-classification/seg_train/seg_train/street/1269.jpg
/kaggle/input/intel-image-classification/seg_train/seg_train/buildings/2193.jpg
/kaggle/input/intel-image-classification/seg_train/seg_train/sea/19812.jpg
/kaggle/input/intel-image-classification/seg_train/seg_train/forest/7981.jpg
/kaggle/input/intel-image-classification/seg_train/seg_train/glacier/12666.jpg
/kaggle/input/intel-image-classification/seg_pred/seg_pred/6234.jpg
/kaggle/input/intel-image-classification/seg_test/seg_test/mountain/22608.jpg
/kaggle/input/intel-image-classification/seg_test/seg_test/street/20088.jpg
/kaggle/input/intel-image-classification/seg_test/seg_test/buildings/22735.jpg
/kaggle/input/intel-image-classification/seg_test/seg_test/sea/20513.jpg
/kaggle/input/intel-image-classification/seg_test/seg_test/forest/23407.jpg
/kaggle/input/intel-image-classification/seg_test/seg_test/glacier/23149.jpg


In [2]:
#Import deep learning libraries

import torch
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, random_split, Subset, Dataset
from sklearn.model_selection import StratifiedShuffleSplit
import numpy as np
from PIL import Image
import time
from tqdm import tqdm


In [3]:
# Define the transformations
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),  # Convert to grayscale
    transforms.Resize((150, 150)),                # Resize images to 150x150
    transforms.ToTensor(),                        # Convert images to tensors
    transforms.Normalize(mean=[0.5], std=[0.5])   # Normalize images to [0, 1]
])

In [4]:
#train dataset
train_path = '/kaggle/input/intel-image-classification/seg_train/seg_train'
train_dataset = datasets.ImageFolder(root=train_path, transform=transform)
# Define batch size
batch_size = 32
# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Example: Iterate over the first batch
for images, labels in train_loader:
    print(images.shape, labels.shape)
    break

torch.Size([32, 1, 150, 150]) torch.Size([32])


In [5]:
#Validation and test data
val_path = '/kaggle/input/intel-image-classification/seg_test/seg_test'
val_dataset_full = datasets.ImageFolder(root=val_path, transform=transform)

targets = np.array(val_dataset_full.targets)  # Labels for stratification
sample_size = 100  # Total sample size

num_iterations = 2  # Number of different samples you want
# Create stratified split with multiple iterations
splitter = StratifiedShuffleSplit(n_splits=num_iterations, test_size=sample_size, random_state=42)

sample_datasets = []
for train_idx, sample_idx in splitter.split(np.zeros(len(targets)), targets):
    sample_datasets.append(Subset(val_dataset_full, sample_idx))

val_dataset = sample_datasets[0]
test_dataset = sample_datasets[1]

# Define batch size
batch_size = 100
# Create DataLoaders
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [ ]:
import numpy as np

class Conv2D:
    """
    2D Convolutional layer with kernel size 3x3, stride 1, padding 1.
    """
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding=1):
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding

        # Initialize weights and biases (He initialization)
        scale = np.sqrt(2.0 / (in_channels * kernel_size * kernel_size))
        self.weight = np.random.randn(out_channels, in_channels, kernel_size, kernel_size) * scale
        self.bias = np.zeros(out_channels)

        # Gradients placeholders
        self.grad_weight = np.zeros_like(self.weight)
        self.grad_bias = np.zeros_like(self.bias)

    def forward(self, x):
        """
        Forward pass: perform convolution.
        x shape: (N, C_in, H, W)
              Returns: out shape: (N, C_out, H_out, W_out)
        """
        self.x = x  # Save input for backward
        N, C, H, W = x.shape
        assert C == self.in_channels, "Input channel dimension mismatch."

        # Pad input
        pad = self.padding
        x_padded = np.pad(x, ((0,0),(0,0),(pad,pad),(pad,pad)), mode='constant', constant_values=0)

        # Compute output dimensions
        out_H = (H + 2*pad - self.kernel_size) // self.stride + 1
        out_W = (W + 2*pad - self.kernel_size) // self.stride + 1

        # Initialize output tensor
        out = np.zeros((N, self.out_channels, out_H, out_W))

        # Perform convolution (naive loop implementation)
        for n in range(N):
            for o in range(self.out_channels):
                for i in range(out_H):
                    for j in range(out_W):
                        i0 = i * self.stride
                        j0 = j * self.stride
                        region = x_padded[n, :, i0:i0+self.kernel_size, j0:j0+self.kernel_size]
                        out[n, o, i, j] = np.sum(region * self.weight[o]) + self.bias[o]
        self.out = out  # Save output (if needed)
        return out

    def backward(self, dout):
        """
        Backward pass: compute gradients w.r.t. input, weights, biases.
        dout shape: (N, C_out, H_out, W_out)
        Returns: dx shape: (N, C_in, H, W)
        """
        x = self.x
        N, C_in, H, W = x.shape
        pad = self.padding
        x_padded = np.pad(x, ((0,0),(0,0),(pad,pad),(pad,pad)), mode='constant', constant_values=0)

        # Initialize gradients
        dx_padded = np.zeros_like(x_padded)
        self.grad_weight = np.zeros_like(self.weight)
        self.grad_bias = np.zeros_like(self.bias)

        # Gradient w.r.t. bias: sum over batch, height and width for each output channel
        self.grad_bias = np.sum(dout, axis=(0,2,3))

        N, C_out, out_H, out_W = dout.shape
        # Compute gradients w.r.t. weights and inputs
        for n in range(N):
            for o in range(C_out):
                for i in range(out_H):
                    for j in range(out_W):
                        i0 = i * self.stride
                        j0 = j * self.stride
                        region = x_padded[n, :, i0:i0+self.kernel_size, j0:j0+self.kernel_size]
                        # Weight gradient accumulation
                        self.grad_weight[o] += region * dout[n, o, i, j]
                        # Input gradient propagation
                        dx_padded[n, :, i0:i0+self.kernel_size, j0:j0+self.kernel_size] += self.weight[o] * dout[n, o, i, j]

        # Remove padding from dx
        if pad > 0:
            dx = dx_padded[:, :, pad:-pad, pad:-pad]
        else:
            dx = dx_padded
        return dx


class BatchNorm2D:
    """
    Batch Normalization layer for 2D convolution outputs.
    """
    def __init__(self, num_features, eps=1e-5):
        self.num_features = num_features
        self.eps = eps
        # Scale and shift parameters
        self.gamma = np.ones(num_features)
        self.beta = np.zeros(num_features)
        # Gradients
        self.grad_gamma = np.zeros_like(self.gamma)
        self.grad_beta = np.zeros_like(self.beta)

    def forward(self, x):
        """
        Forward batch norm.
        x shape: (N, C, H, W)
        """
        self.x = x
        N, C, H, W = x.shape
        # Flatten spatial dimensions for computing mean/var per channel
        x_flat = x.transpose(1, 0, 2, 3).reshape(C, -1)

        # Compute mean and variance per channel
        self.mean = np.mean(x_flat, axis=1)
        self.var = np.var(x_flat, axis=1)
        self.std = np.sqrt(self.var + self.eps)

        # Normalize
        x_norm_flat = (x_flat - self.mean[:, None]) / self.std[:, None]
         # Reshape back to original
        x_norm = x_norm_flat.reshape(C, N, H, W).transpose(1, 0, 2, 3)

        # Scale and shift
        out = self.gamma[None, :, None, None] * x_norm + self.beta[None, :, None, None]
        self.x_norm = x_norm  # Save normalized input for backward
        return out

    def backward(self, dout):
        """
        Backward pass for batch norm.
        dout shape: (N, C, H, W)
        """
        N, C, H, W = dout.shape
        # Reshape to (C, N*H*W)
        dout_flat = dout.transpose(1, 0, 2, 3).reshape(C, -1)
        x_flat = self.x.transpose(1, 0, 2, 3).reshape(C, -1)

        # Gradients for gamma and beta
        x_norm_flat = self.x_norm.transpose(1, 0, 2, 3).reshape(C, -1)
        self.grad_gamma = np.sum(dout_flat * x_norm_flat, axis=1)
        self.grad_beta = np.sum(dout_flat, axis=1)

        # Gradient w.r.t. normalized input
        dx_norm = (dout * self.gamma[None, :, None, None]).transpose(1, 0, 2, 3).reshape(C, -1)

        # Intermediate terms for backward pass
        m = N * H * W
        x_mu = x_flat - self.mean[:, None]

        # Gradients w.r.t. variance and mean
        dvar = np.sum(dx_norm * x_mu * -0.5 * (self.var[:, None] + self.eps)**(-1.5), axis=1)
        dmean = np.sum(dx_norm * -1.0 / self.std[:, None], axis=1) + dvar * np.mean(-2.0 * x_mu, axis=1)

        # Gradient w.r.t. input
        dx_flat = (dx_norm / self.std[:, None]) + (dvar[:, None] * 2 * x_mu / m) + (dmean[:, None] / m)
        dx = dx_flat.reshape(C, N, H, W).transpose(1, 0, 2, 3)
        return dx


class ReLU:
    """
    ReLU activation layer.
    """
    def forward(self, x):
        self.mask = (x > 0)
        return x * self.mask

    def backward(self, dout):
        return dout * self.mask


class MaxPool2D:
    """
    Max pooling layer with kernel size 2x2 and stride 2.
    """
    def __init__(self, kernel_size=2, stride=2):
        self.kernel_size = kernel_size
        self.stride = stride

    def forward(self, x):
        """
        x shape: (N, C, H, W)
        Returns: pooled output of shape (N, C, H_out, W_out).
        """
        self.x = x
        N, C, H, W = x.shape
        pool = self.kernel_size
        stride = self.stride
        # Compute output dimensions
        out_H = (H - pool) // stride + 1
        out_W = (W - pool) // stride + 1

        out = np.zeros((N, C, out_H, out_W))
        self.mask = np.zeros_like(x)  # mask for max indices

        for n in range(N):
            for c in range(C):
                for i in range(out_H):
                    for j in range(out_W):
                        i0 = i * stride
                        j0 = j * stride
                        region = x[n, c, i0:i0+pool, j0:j0+pool]
                        max_val = np.max(region)
                        out[n, c, i, j] = max_val
                        # Mask the location of max value for backward
                        max_idx = np.unravel_index(np.argmax(region), region.shape)
                        self.mask[n, c, i0+max_idx[0], j0+max_idx[1]] = 1
        return out

    def backward(self, dout):
        """
        Backpropagate through max pool.
        dout shape: (N, C, H_out, W_out)
        """
        N, C, out_H, out_W = dout.shape
        pool = self.kernel_size
        stride = self.stride
        dx = np.zeros_like(self.x)

        for n in range(N):
            for c in range(C):
                for i in range(out_H):
                    for j in range(out_W):
                        i0 = i * stride
                        j0 = j * stride
                        # Propagate gradient to the max location
                        dx[n, c, i0:i0+pool, j0:j0+pool] += \
                            self.mask[n, c, i0:i0+pool, j0:j0+pool] * dout[n, c, i, j]
        return dx


class Flatten:
    """
    Flatten layer to convert 4D tensor to 2D.
    """
    def forward(self, x):
        self.orig_shape = x.shape
        N = x.shape[0]
        return x.reshape(N, -1)

    def backward(self, dout):
        return dout.reshape(self.orig_shape)


class Linear:
    """
    Fully connected linear layer.
    """
    def __init__(self, in_features, out_features):
        self.in_features = in_features
        self.out_features = out_features
        # Xavier/He initialization
        scale = np.sqrt(2.0 / in_features)
        self.weight = np.random.randn(in_features, out_features) * scale
        self.bias = np.zeros(out_features)
        self.grad_weight = np.zeros_like(self.weight)
        self.grad_bias = np.zeros_like(self.bias)

    def forward(self, x):
        """
        Forward pass for linear layer.
        x shape: (N, in_features)
        returns shape: (N, out_features)
        """
        self.x = x  # save input
        return x.dot(self.weight) + self.bias

    def backward(self, dout):
        """
        Backward pass for linear layer.
        dout shape: (N, out_features)
        """
        # Gradient w.r.t. weights and bias
        self.grad_weight = self.x.T.dot(dout)
        self.grad_bias = np.sum(dout, axis=0)
        # Gradient w.r.t. input
        dx = dout.dot(self.weight.T)
        return dx
def softmax_cross_entropy_with_logits(logits, labels):
    """
    Compute softmax probabilities and cross-entropy loss.
    logits: (N, C)
    labels: (N,) with class indices 0..C-1
    Returns: loss (scalar) and softmax probabilities (N, C).
    """
    # Shift logits for numerical stability
    logits_shift = logits - np.max(logits, axis=1, keepdims=True)
    exp_scores = np.exp(logits_shift)
    softmax = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)
    N = logits.shape[0]
    # Cross-entropy loss
    correct_logprobs = -np.log(softmax[np.arange(N), labels] + 1e-12)
    loss = np.sum(correct_logprobs) / N
    return loss, softmax


def softmax_grad(softmax, labels):
    """
    Gradient of softmax + cross-entropy loss w.r.t. logits.
    """
    N = softmax.shape[0]
    grad = softmax.copy()
    grad[np.arange(N), labels] -= 1
    grad = grad / N
    return grad


class SimpleCNN:
    """
    CNN architecture:
    Conv(1->16) -> BN -> ReLU -> Pool ->
    Conv(16->32) -> BN -> ReLU -> Pool ->
    Flatten -> FC(256) -> ReLU -> FC(6)
    """
    def __init__(self):
        # Layer definitions
        self.conv1 = Conv2D(1, 16, kernel_size=3, stride=1, padding=1)
        self.bn1   = BatchNorm2D(16)
        self.relu1 = ReLU()
        self.pool1 = MaxPool2D(2, 2)

        self.conv2 = Conv2D(16, 32, kernel_size=3, stride=1, padding=1)
        self.bn2   = BatchNorm2D(32)
        self.relu2 = ReLU()
        self.pool2 = MaxPool2D(2, 2)

        self.flatten = Flatten()

        # Calculate flattened feature size after conv/pool 
        flat_features = 32 * 37 * 127
        self.fc1 = Linear(43808, 256)
        self.relu3 = ReLU()
        self.fc2 = Linear(256, 6)

    def forward(self, x):
        # Pass through conv -> BN -> ReLU -> Pool blocks
        out = self.conv1.forward(x)
        out = self.bn1.forward(out)
        out = self.relu1.forward(out)
        out = self.pool1.forward(out)

        out = self.conv2.forward(out)
        out = self.bn2.forward(out)
        out = self.relu2.forward(out)
        out = self.pool2.forward(out)

        # Flatten and fully connected layers
        out = self.flatten.forward(out)
        out = self.fc1.forward(out)
        out = self.relu3.forward(out)
        out = self.fc2.forward(out)
        return out
    
    def backward(self, dout):
        # Backpropagate through layers in reverse order
        dout = self.fc2.backward(dout)
        dout = self.relu3.backward(dout)
        dout = self.fc1.backward(dout)
        dout = self.flatten.backward(dout)

        dout = self.pool2.backward(dout)
        dout = self.relu2.backward(dout)
        dout = self.bn2.backward(dout)
        dout = self.conv2.backward(dout)

        dout = self.pool1.backward(dout)
        dout = self.relu1.backward(dout)
        dout = self.bn1.backward(dout)
        dout = self.conv1.backward(dout)

        return dout

    def update_params(self, lr=0.001):
        """
        Update all parameters using SGD with learning rate lr.
        """
        # Conv1 parameters
        self.conv1.weight -= lr * self.conv1.grad_weight
        self.conv1.bias   -= lr * self.conv1.grad_bias
        # BN1 parameters
        self.bn1.gamma -= lr * self.bn1.grad_gamma
        self.bn1.beta  -= lr * self.bn1.grad_beta
        # Conv2 parameters
        self.conv2.weight -= lr * self.conv2.grad_weight
        self.conv2.bias   -= lr * self.conv2.grad_bias
        # BN2 parameters
        self.bn2.gamma -= lr * self.bn2.grad_gamma
        self.bn2.beta  -= lr * self.bn2.grad_beta
        # FC1 parameters
        self.fc1.weight -= lr * self.fc1.grad_weight
        self.fc1.bias   -= lr * self.fc1.grad_bias
        # FC2 parameters
        self.fc2.weight -= lr * self.fc2.grad_weight
        self.fc2.bias   -= lr * self.fc2.grad_bias

In [ ]:
net = SimpleCNN()

# Hiperparámetros
num_epochs = 10
learning_rate = 0.001

# Entrenamiento del modelo
for epoch in range(num_epochs):
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    for images, labels in train_loader:
        images, labels = images.numpy(), labels.numpy()

        # Forward pass
        logits = net.forward(images)
        loss, probs = softmax_cross_entropy_with_logits(logits, labels)
        running_loss += loss

        # Cálculo de precisión
        predictions = np.argmax(probs, axis=1)
        correct_train += np.sum(predictions == labels)
        total_train += labels.shape[0]

        # Backward pass
        grad_logits = softmax_grad(probs, labels)
        net.backward(grad_logits)

        # Actualización de parámetros
        net.update_params(learning_rate)

    train_accuracy = correct_train / total_train

    # Validación del modelo
    correct_val = 0
    total_val = 0
    val_loss = 0

    for images, labels in val_loader:
        images, labels = images.numpy(), labels.numpy()

        # Forward pass
        logits = net.forward(images)
        loss, probs = softmax_cross_entropy_with_logits(logits, labels)
        val_loss += loss

        # Cálculo de precisión
        predictions = np.argmax(probs, axis=1)
        correct_val += np.sum(predictions == labels)
        total_val += labels.shape[0]
    
    val_accuracy = correct_val / total_val
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, Validation Accuracy: {val_accuracy:.4f}")

Epoch [1/10], Loss: 597.5251, Train Accuracy: 0.6180, Validation Accuracy: 0.6100
Epoch [2/10], Loss: 284.8785, Train Accuracy: 0.7604, Validation Accuracy: 0.8100
Epoch [3/10], Loss: 253.9044, Train Accuracy: 0.7857, Validation Accuracy: 0.7600
Epoch [4/10], Loss: 228.9225, Train Accuracy: 0.8090, Validation Accuracy: 0.7800
Epoch [5/10], Loss: 212.5150, Train Accuracy: 0.8234, Validation Accuracy: 0.7300
Epoch [6/10], Loss: 186.9963, Train Accuracy: 0.8425, Validation Accuracy: 0.7800
Epoch [7/10], Loss: 163.5944, Train Accuracy: 0.8622, Validation Accuracy: 0.7500
Epoch [8/10], Loss: 146.3313, Train Accuracy: 0.8809, Validation Accuracy: 0.7800
Epoch [9/10], Loss: 125.9772, Train Accuracy: 0.8974, Validation Accuracy: 0.7700
Epoch [10/10], Loss: 111.6245, Train Accuracy: 0.9057, Validation Accuracy: 0.7600
